In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [4]:
df = pd.read_csv("model_variables.csv")
df.shape

C:\Users\27311\AppData\Local\Temp\ipykernel_18276\1781622380.py:1: DtypeWarning: Columns (48,49,50,51,57,58,59,60,66,67,68,69,75,76,77,78,84,85,86,87,92,93,94,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("model_variables.csv")


(701968, 106)

In [8]:
# === 2. Keep only records with at least one error ===
err = df[df['error_flag'] == 1].copy()

print(err.shape)
print(err['element1'].value_counts().head(20))  # top 20 error elements


(95977, 106)
element1
wages and salaries                                    24609
shelter deduction                                     23474
standard utility allowance                             6656
rsdi benefits                                          6243
child support payments received from absent parent     5000
unit composition                                       4382
medical expense deductions                             4126
ssi and/or state ssi supplement                        3727
self-employment                                        3044
dependent care deduction                               2066
other unearned income                                  1814
arithmetic computation                                 1752
unemployment compensation                              1747
contributions                                          1421
child support payment deduction                        1206
tanf, pa, or ga                                         955
student status    

In [12]:
# === 3. Recode element1 into top K categories + "Other" ===
K = 15  # you can change to 15 if you want more classes

value_counts = err['element1'].value_counts()
top_elements = value_counts.head(K).index

def recode_element(x):
    # keep top K elements, all others go to "Other element"
    if x in top_elements:
        return x
    else:
        return "Other element"

err['elem_cat'] = err['element1'].map(recode_element)

print(err['elem_cat'].value_counts())


elem_cat
wages and salaries                                    24609
shelter deduction                                     23474
standard utility allowance                             6656
rsdi benefits                                          6243
child support payments received from absent parent     5000
Other element                                          4710
unit composition                                       4382
medical expense deductions                             4126
ssi and/or state ssi supplement                        3727
self-employment                                        3044
dependent care deduction                               2066
other unearned income                                  1814
arithmetic computation                                 1752
unemployment compensation                              1747
contributions                                          1421
child support payment deduction                        1206
Name: count, dtype: int64


In [38]:
# === 4. Define features (X variables) ===

# Continuous / numeric variables
features_cont = [
    'fsusize',    # household size
    'fsnetinc',   # net income
    'tpov',       # poverty ratio
    'fstotde2'    # monthly SNAP benefit
]

# Binary variables (0/1)
features_bin = [
    'children_present',  # 1 = children in HH, 0 = no
    'elderly_present',   # 1 = elderly in HH
    'fsndis',            # disability
    'abwdst1'            # ABAWD status flag
]

# Categorical variables
features_cat = [
    'actntype',   # action type (initial, recert, etc.)
    'cat_elig'
    # ,   # categorical eligibility
    # # 'state.1',    # state
    # # 'year'        # year (treat as categorical for now)
]

# Combine all predictors
features = features_cont + features_bin + features_cat


In [40]:
# === 5. Keep only needed columns and drop rows with missing values ===
cols_needed = ['elem_cat'] + features

# This will keep only rows with no missing values in selected columns
err_model = err[cols_needed].dropna().copy()

print(err_model.shape)
print(err_model['elem_cat'].value_counts())


(94900, 11)
elem_cat
wages and salaries                                    24471
shelter deduction                                     23036
standard utility allowance                             6553
rsdi benefits                                          6215
child support payments received from absent parent     4984
Other element                                          4568
unit composition                                       4324
medical expense deductions                             4124
ssi and/or state ssi supplement                        3667
self-employment                                        3021
dependent care deduction                               2065
other unearned income                                  1793
unemployment compensation                              1743
arithmetic computation                                 1726
contributions                                          1411
child support payment deduction                        1199
Name: count, dtype:

In [42]:
# === 6. One-hot encoding for categorical variables ===
X = pd.get_dummies(
    err_model[features],
    columns=features_cat,
    drop_first=True  # drop first level to avoid perfect multicollinearity
)

y = err_model['elem_cat']

print(X.shape)
print(y.head())


(94900, 11)
15    standard utility allowance
44      dependent care deduction
48             shelter deduction
50    standard utility allowance
52        arithmetic computation
Name: elem_cat, dtype: object


In [44]:
# === 7. Encode y as categorical codes ===
y_cat = y.astype('category')
y_codes = y_cat.cat.codes  # 0,1,2, ...

# mapping from numeric code to original category
label_map = dict(enumerate(y_cat.cat.categories))
print("Label map (code -> element):")
print(label_map)


Label map (code -> element):
{0: 'Other element', 1: 'arithmetic computation', 2: 'child support payment deduction', 3: 'child support payments received from absent parent', 4: 'contributions', 5: 'dependent care deduction', 6: 'medical expense deductions', 7: 'other unearned income', 8: 'rsdi benefits', 9: 'self-employment', 10: 'shelter deduction', 11: 'ssi and/or state ssi supplement', 12: 'standard utility allowance', 13: 'unemployment compensation', 14: 'unit composition', 15: 'wages and salaries'}


In [46]:
# === Fix: Convert boolean columns to integers (True=1, False=0) ===
X = X.astype(float)   # this forces bool -> 1.0/0.0 and int stays int


In [50]:
from sklearn.preprocessing import StandardScaler

err_model = err[['elem_cat'] + features].dropna().copy()

# 1) scale continuous vars
scaler = StandardScaler()
X_num = err_model[features_cont + features_bin].copy()
X_num[features_cont] = scaler.fit_transform(X_num[features_cont])

# 2) one-hot categorical vars
X_cat = pd.get_dummies(
    err_model[features_cat],
    drop_first=True
)

# 3) combine
X = pd.concat([X_num, X_cat], axis=1)

# 4) force float
X = X.astype(float)

# y
y = err_model['elem_cat'].astype('category')
y_codes = y.cat.codes
label_map = dict(enumerate(y.cat.categories))
print(label_map)


{0: 'Other element', 1: 'arithmetic computation', 2: 'child support payment deduction', 3: 'child support payments received from absent parent', 4: 'contributions', 5: 'dependent care deduction', 6: 'medical expense deductions', 7: 'other unearned income', 8: 'rsdi benefits', 9: 'self-employment', 10: 'shelter deduction', 11: 'ssi and/or state ssi supplement', 12: 'standard utility allowance', 13: 'unemployment compensation', 14: 'unit composition', 15: 'wages and salaries'}


In [57]:
for code, label in label_map.items():
    print(f"y={code}  →  {label}")

y=0  →  Other element
y=1  →  arithmetic computation
y=2  →  child support payment deduction
y=3  →  child support payments received from absent parent
y=4  →  contributions
y=5  →  dependent care deduction
y=6  →  medical expense deductions
y=7  →  other unearned income
y=8  →  rsdi benefits
y=9  →  self-employment
y=10  →  shelter deduction
y=11  →  ssi and/or state ssi supplement
y=12  →  standard utility allowance
y=13  →  unemployment compensation
y=14  →  unit composition
y=15  →  wages and salaries


In [52]:
import statsmodels.api as sm

X_const = sm.add_constant(X)

model = sm.MNLogit(y_codes, X_const)
result = model.fit(method='lbfgs', maxiter=300)  
print(result.summary())


                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                94900
Model:                        MNLogit   Df Residuals:                    94735
Method:                           MLE   Df Model:                          150
Date:                Sat, 29 Nov 2025   Pseudo R-squ.:                  0.1406
Time:                        23:14:41   Log-Likelihood:            -1.8691e+05
converged:                       True   LL-Null:                   -2.1750e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                     y=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -0.4989      0.119     -4.179      0.000      -0.733      -0.265
fsusize                     -0.6114      0.072     -8.458      0.000      -0.753      -0.